# LLM Fine-Tuning (gemma-2b + LoRA)

공개된 대규모 언어모델 **"gemma-2b"** 을 Foundation Model로 하고, LoRA 기법을 적용하여 효과적으로 파인튜닝하는 실습을 진행해 보겠습니다.  
- Task: Causal Language Model (Instruct Fine-Tuning)
- Foundation Model: “gemma-2b”
- Dataset: “KorAlpaca”
- Trainer: Huggingface PEFT/LoRA

## 0. Setup

In [1]:
# MLP Suwon 설정 필요
import os

os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'
os.environ['HTTP_PROXY'] ='http://75.17.107.42:8080'
os.environ['HTTPS_PROXY'] ='http://75.17.107.42:8080'

In [2]:
# MLP Suwon 설정 필요
import ssl

if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# !pip install -q --user transformers==4.38.2
# !pip install -q --user datasets==2.18.0
# !pip install -q --user peft==0.9.0
# !pip install -q --user trl==0.7.11
# !pip install -q --user accelerate==0.27.2


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [4]:
!nvidia-smi

Tue Oct 22 13:57:25 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   33C    P8    10W / 250W |     10MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 1. DataSet: KoAlpaca v1.1a

질문을 전달하면 답변하는 형태로 대규모 언어모델을 파인튜닝하기 위한 학습 데이터셋을 준비하겠습니다.  
데이터셋은 **Instruction** (지시사항)과 **Response** (출력)의 쌍으로 구성되어 있습니다.  
KoAlpaca 데이터셋은 지식iN 기반의 질문-답변 데이터셋이며, Huggingface Dataset 에 공개되어 있습니다.

In [5]:
from datasets import load_dataset

# dataset = load_dataset("beomi/KoAlpaca-v1.1a")
dataset = load_dataset("/group-volume/sr_edu/AI-Application-Specialist/LLM/dataset/KoAlpaca-v1.1a")

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [6]:
dataset['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

파인튜닝하기 위하여 학습 데이터를 다음과 같은 Instruction Format 형태로 변환을 합니다.  
```
### Question:

### Response: <eos>
```

In [7]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        prompt = f"### Question: {example['instruction'][i]}\n\n### Response: {example['output'][i]}<eos>"
        output_texts.append(prompt)
    return output_texts

In [8]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

### Question: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<eos>


## 2. Foundation Model & Tokenizer

In [9]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Foundation LLM 으로 "google/gemma-2b" 모델을 로딩합니다.  
- Lightweight, state-of-the-art open models
- Text-to-text, decoder-only LLM (English)
- Instruction following & Multi-turn conversations

In [10]:
# BASE_MODEL = "google/gemma-2b"
BASE_MODEL = "/group-volume/sr_edu/AI-Application-Specialist/LLM/model/gemma-2b"

# [실습] 다음 코드를 완성하세요!! 
# 사전 학습된 'google/gemma-2b' 모델과 토크나이저를 가져옵니다.
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
prompt = "건강하게 살기 위한 세 가지 방법은?"

In [12]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)

print(outputs[0]["generated_text"][len(prompt):])



1. <strong>운동</strong>
2. <strong>식사</strong>
3. <strong>정신적 안녕감</strong>

이 중에서도 운동을 가장 중요시하는 이유는 무엇일까요?

* 운동은 체력과 건강을 향상시키고, 심장질환이나 고혈압 등의 질병을 예방한다.
* 운동은 정신적인 안녕감을 높여준다.
* 운동은 신체적 안녕감을 높여준다.

그러므로, 우리가 건강한 생활을 하기 위해서는 운동을 하는 것이 매우 중요하다고 할 수 있겠습니다.

<h2>운동에 대한 이해</h2>

우선, 운동이란 무엇인지 알아보도록 하겠습니다.

<h3>무엇을 의미하는 ‘운동’</h3>

‘운동’이라는 단어는 여러 가지 의미를 지니고 있습니다.

* 운동은 사람이 어떤 일을 하면서 그 일을 몸으로 움직이는 것을 말합니다.
* 운동은 사람이 몸을 활용하여 일을 하거나, 힘든 일을 해결하기 위해 몸을 사용하는 것을 말합니다.
* 운동은 사람이 몸을 활용하여 일을 하거나, 힘든 일을 해결하기 위해 몸을 사용하는 것을 말합니다.

위와 같은 의미로 사용되고 있는데, 이 중에서도 <strong>“자연스럽게”</strong> 하는 의미를 지닌 운동은 다음과 같습니다.

<blockquote>자연스럽게 = 자연스럽게 하는 것은 자연스럽게 하는 것</blockquote>


## 3. PEFT LoRA Setup

LoraConfig 함수를 통해 파인튜닝을 위한 주요 LoRA 파라미터를 설정합니다.

In [13]:
# [실습] 다음 코드를 완성하세요!! 
# LoRA 파인튜닝을 위한 Config를 설정합니다. r, lora_alpha, lora_dropout, target_modules, task_type
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
"""

## 4. Model Training (PEFT LoRA)

SFT Trainer 관련 필요한 학습 파라미터 설정한 후 PEFT LoRA 기반으로 파인튜닝을 위한 학습을 진행합니다.

In [14]:
# [실습] 다음 코드를 완성하세요!! 
# 파인튜닝을 위한 Trainer 설정을 합니다. (model, train_dataset, max_seq_length, args, peft_config, generate_func) 
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=600, # 3000
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="adamw_hf",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=200,
        push_to_hub=False,
        report_to='none',
        dataloader_num_workers=4,
        dataloader_prefetch_factor=2,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/user/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

LoRA 모델을 통해 학습 가능한 파라미터수는 전체 모델 파라미터(2,513,526,784)의 0.5% 수준입니다.

In [16]:
print_trainable_parameters(model)

trainable params: 1382400 || all params: 2507554816 || trainable%: 0.05512940300165307


600 스텝 학습에 25분 정도 소요되니, TrainingArguments에서 **`max_steps`** 설정하실 때 참고하시기 바랍니다.

In [17]:
trainer.train()

/home/user/.local/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,1.905100
400,1.846100
600,1.845600


/home/user/.local/lib/python3.8/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /group-volume/sr_edu/AI-Application-Specialist/LLM/model/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=600, training_loss=1.8655912780761719, metrics={'train_runtime': 1271.3819, 'train_samples_per_second': 1.888, 'train_steps_per_second': 0.472, 'total_flos': 8353496029790208.0, 'train_loss': 1.8655912780761719, 'epoch': 0.11})

학습된 LoRA Adapter 모델을 Local Directory 에 저장합니다.

In [ ]:
"""
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)
"""

LoRA 학습된 weight을 원래 gemma-2b 모델과 합쳐 하나의 모델로 만들겠습니다.

In [ ]:
"""
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
# model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)
model.load_adapter(ADAPTER_MODEL)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-peft')
"""

## 5. Model Inference

파인튜닝된 모델을 이용하여 Generation 해 보도록 하겠습니다.

In [18]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [19]:
prompt = "건강하게 살기 위한 세 가지 방법은?"
formatted_prompt = f"### Question: {prompt}\n\n### Response:"

In [20]:
outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 1) 운동을 하며, 2) 음식물의 양과 질을 관리하며, 3) 스트레스를 줄이는 것.


- Ref. https://huggingface.co/docs/peft/main/en/task_guides/token-classification-lora